Code Credit - https://twitter.com/pwang_szn/status/1642104548109201410/photo/1


In [1]:
!pip install -q langchain openai chromadb
!pip install tiktoken
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.7/507.7 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade openai


  Using cached openai-1.6.0-py3-none-any.whl (225 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import openai
import os

In [4]:
os.environ["OPENAI_API_KEY"] = "api"

In [15]:
import csv
import yfinance as yf
msft = yf.Ticker("goog")

# get historical market data
hist = msft.history(period="4mo")
hist=hist.reset_index()
# Open a file in write mode
hist.to_csv('filename.csv', index=False)
# Load the documents
loader = CSVLoader(file_path='/content/filename.csv')

In [16]:
# Create an index using the loaded documents
index_creator = VectorstoreIndexCreator()
docsearch = index_creator.from_loaders([loader])
docsearch.vectorstore.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7e69a4d100d0>)

In [17]:
# Create a question-answering chain using the index
chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")


In [23]:
# Pass a query to the chain
query = "can you give me describtion of the data"
response = chain({"question": query})
dataset_des =response['result']
dataset_des

' The data is for stocks on the dates 2023-08-29 and 2023-10-02. On 2023-08-29, the open was 132.9980010986328, the high was 137.2949981689453, the low was 132.97999572753906, the close was 135.49000549316406, and the volume was 30803300. On 2023-10-02, the open was 132.15499877929688, the high was 135.36000061035156, the low was 132.06500244140625, the close was 135.1699981689453, and the volume was 19210400. There were no dividends or stock splits for either date.'

In [21]:
# Pass a query to the chain
query = "average open from  2023-09-11 to 2023-09-16"
response = chain({"question": query})
response['result']


" I don't know."

In [ ]:
# Pass a query to the chain
query = "average open from  2023-09-11 to 2023-09-20"
response = chain({"question": query})
response['result']
